### Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
import IPython.display as display_obj
from random import randint


import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
import os

##Download Images

Download images and choose a style image and a content image:

In [ ]:
#https://cdn.pixabay.com/photo/2017/02/28/23/00/swan-2107052_1280.jpg

!wget  https://cdn.pixabay.com/photo/2018/07/14/15/27/cafe-3537801_1280.jpg
!wget  https://cdn.pixabay.com/photo/2017/02/28/23/00/swan-2107052_1280.jpg
!wget  https://i.dawn.com/large/2019/10/5db6a03a4c7e3.jpg
!wget  https://cdn.pixabay.com/photo/2015/09/22/12/21/rudolph-951494_1280.jpg
!wget https://cdn.pixabay.com/photo/2015/10/13/02/59/animals-985500_1280.jpg

_, content_path = os.path.split("https://cdn.pixabay.com/photo/2018/07/14/15/27/cafe-3537801_1280.jpg")
_, style_path = os.path.split("https://cdn.pixabay.com/photo/2015/09/22/12/21/rudolph-951494_1280.jpg")
#style_path = tf.keras.utils.get_file('style_image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

## Visualize the input

Define a function to load an image and limit its maximum dimension to 512 pixels.

In [ ]:
def tensor_to_image(tensor):
  tensor_shape = tf.shape(tensor)
  number_elem_shape = tf.shape(tensor_shape)
  if number_elem_shape > 3:
    assert tensor_shape[0] == 1
    tensor = tensor[0]
  return tf.keras.preprocessing.image.array_to_img(tensor) 

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  image = tf.io.read_file(path_to_img)
  image = tf.image.decode_jpeg(image)
  image = tf.image.convert_image_dtype(image, tf.float32)

  shape = tf.shape(image)[:-1]
  shape = tf.cast(tf.shape(image)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  image = tf.image.resize(image, new_shape)
  image = image[tf.newaxis, :]
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return image

In [ ]:
def preprocess_image(image):
    image = tf.cast(image, dtype=tf.float32)
    image = tf.keras.applications.vgg19.preprocess_input(image)

    return image


Create a simple function to display an image:

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
def show_images_with_objects(images, titles=[]):

  if len(images) != len(titles):
    return

  plt.figure(figsize=(20, 12))
  for idx, (image, title) in enumerate(zip(images, titles)):
    plt.subplot(1, len(images), idx + 1)
    plt.xticks([])
    plt.yticks([])
    imshow(image, title)
    

In [ ]:
def load_images(content_path, style_path):
  content_image = load_img("{}".format(content_path))
  style_image = load_img("{}".format(style_path))

  return content_image, style_image

## Build the model 

Choose intermediate layers from the network to represent the style and content of the image:


In [ ]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer of interest
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [ ]:
def vgg_model(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  print(vgg.input)
  model = tf.keras.Model(inputs=vgg.input, outputs=outputs)
  return model

vgg = vgg_model(style_layers + content_layers)

And to create the model:

In [ ]:
def get_style_loss(features, targets):
  # """Expects two images of dimension h, w, c"""
  # # height, width, num filters of each layer
  # # We scale the loss at a given layer by the size of the feature map and the number of filters
  # height, width, channels = base_style.get_shape().as_list()
  # gram_style = gram_matrix(base_style)
  return tf.reduce_mean(tf.square(features - targets))

In [ ]:
def get_content_loss(features, targets):
  return tf.reduce_mean(tf.square(features - targets))

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

In [ ]:
def get_style_image_features(image):  
  preprocessed_style_image = preprocess_image(image)
  style_outputs = vgg(preprocessed_style_image)#
  gram_style_features = []
  gram_style_features = [gram_matrix(style_layer) for style_layer in style_outputs[:num_style_layers]]
  #style_features = [tf.reshape(style_layer, shape=tf.shape(style_layer)[1:]) for style_layer in style_outputs[:num_style_layers]]
  #style_features = gram_matrix(style_features)
  return gram_style_features

In [ ]:
def get_content_image_features(image):

  preprocessed_content_image = preprocess_image(image)
  vgg_outputs = vgg(preprocessed_content_image)#

  content_features = [content_layer for content_layer in vgg_outputs[num_style_layers:]]

  return content_features

In [ ]:
def get_style_content_loss(style_targets, style_outputs, content_targets, content_outputs, style_weight, content_weight):
  style_loss = tf.add_n([ get_style_loss(style_output, style_target)
                           for style_output, style_target in zip(style_outputs, style_targets)])
  style_loss *= style_weight / num_style_layers

  content_loss = tf.add_n([get_content_loss(content_output, content_target)
                           for content_output, content_target in zip(content_outputs, content_targets)])
  content_loss *= content_weight / num_content_layers
  loss = style_loss + content_loss
  return loss

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
def calculate_gradients(image, content_targets, style_targets, style_weight, content_weight,with_regularization=False ):
    total_variation_weight = 30

    with tf.GradientTape() as tape:
      style_features = get_style_image_features(image * 255)
      content_features = get_content_image_features(image * 255)
      loss = get_style_content_loss(style_targets, style_features, content_targets, content_features, style_weight, content_weight)
      if with_regularization:
        loss += total_variation_weight*tf.image.total_variation(image)

    gradients = tape.gradient(loss, image)
    return gradients

In [ ]:
def update_image_with_style(image, content_targets, style_targets, optimizer, style_weight, content_weight, with_regularization=False):
  
  gradients = calculate_gradients(image, content_targets, style_targets, style_weight, content_weight, with_regularization)
  optimizer.apply_gradients([(gradients, image)])
  image.assign(clip_0_1(image))

##Train Model

In [ ]:
def fit_style_transfer(input_image, style_image, optimizer, epochs=1, steps_per_epoch=1, with_regularization=False, style_weight = 0.01):

  images = []
  import time
  start = time.time()

  step = 0

  #style_weight=1.0
  content_weight=1e2

  style_targets = get_style_image_features(style_image)
  content_targets = get_content_image_features(input_image)


  input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
  
  input_image = tf.Variable(input_image) 
  images.append(tf.Variable(input_image)) 
  
  for n in range(epochs):
    for m in range(steps_per_epoch):
      step += 1
      update_image_with_style(input_image, content_targets, style_targets, optimizer, style_weight, content_weight, with_regularization=with_regularization)

      print(".", end='')
      if (m + 1) % 10 == 0:
        images.append(tf.Variable(input_image))
    
    display_obj.clear_output(wait=True)
    display_image = tensor_to_image(input_image)

    
    display_obj.display(display_image)
    images.append(tf.Variable(input_image))
    print("Train step: {}".format(step))
  end = time.time()
  print("Total time: {:.1f}".format(end-start)) 
  
  return input_image, images

In [ ]:

content_image, style_image = load_images("swan-2107052_1280.jpg", "animals-985500_1280.jpg")


In [ ]:
weight =  0.001 #@param {type:"number"}
adam = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
stylized_image, display_images = fit_style_transfer(input_image=content_image, style_image=style_image, optimizer=adam, epochs=10, steps_per_epoch=100, style_weight=weight)

In [ ]:
#@title (RUN ME!) Display Utilities

import imageio
from IPython.display import display as display_fn
from IPython.display import Image

def display_gif(GIF_PATH):
  with open(GIF_PATH,'rb') as f:
    display_fn(Image(data=f.read(), format='png'))

def create_gif(images):
  GIF_PATH = "/content/{}.gif".format(randint(0, 10000))
  imageio.mimsave(GIF_PATH, images, fps=1)
  return GIF_PATH


In [ ]:
#@title (RUN ME!) Display GIF of Intermedite Outputs
gif_images = [np.squeeze(image.numpy(), axis=0) for image in display_images]
gif_path = create_gif(gif_images)

In [ ]:
display_gif(gif_path)

## Total variation loss

One downside to this basic implementation is that it produces a lot of high frequency artifacts. Decrease these using an explicit regularization term on the high frequency components of the image. In style transfer, this is often called the *total variation loss*:

In [ ]:
#@title (RUN ME!)Plot Utilities
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

def plot_deltas_for_single_image(x_deltas, y_deltas, name="Original", row=1):
  plt.figure(figsize=(14,10))
  plt.subplot(row,2,1)
  plt.yticks([])
  plt.xticks([])

  imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: {}".format(name))

  plt.subplot(row,2,2)
  plt.yticks([])
  plt.xticks([])
  
  imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: {}".format(name))

def plot_deltas(original_image_deltas, stylized_image_deltas):
  orig_x_deltas, orig_y_deltas = original_image_deltas
  
  stylized_x_deltas, stylized_y_deltas = stylized_image_deltas

  plot_deltas_for_single_image(orig_x_deltas, orig_y_deltas, name="Original")
  plot_deltas_for_single_image(stylized_x_deltas, stylized_y_deltas, name="Stylized Image", row=2)

In [ ]:
#@title (RUN ME!)Display Frequency Variations

original_x_deltas, original_y_deltas = high_pass_x_y(tf.image.convert_image_dtype(content_image, dtype=tf.float32))
stylized_image_x_deltas, stylized_image_y_deltas = high_pass_x_y(stylized_image)

plot_deltas((original_x_deltas, original_y_deltas), (stylized_image_x_deltas, stylized_image_y_deltas))

## Re-run the optimization

Choose a weight for the `total_variation_loss`:

In [ ]:
variation_model_weight =   0.001#@param {type:"number"}

stylized_image1, display_images1 = fit_style_transfer(input_image=content_image, style_image=style_image, optimizer=adam, epochs=10, steps_per_epoch=100, with_regularization=True, style_weight=variation_model_weight)

In [ ]:
#@title (RUN ME!)Display GIF

gif_images1 = [np.squeeze(image.numpy(), axis=0) for image in display_images1]
gif_path1 = create_gif(gif_images1)
display_gif(gif_path1)

In [ ]:
#@title (RUN ME!)Display Frequency Variations

original_x_deltas, original_y_deltas = high_pass_x_y(tf.image.convert_image_dtype(content_image, dtype=tf.float32))
stylized_image_x_deltas, stylized_image_y_deltas = high_pass_x_y(stylized_image)

plot_deltas((original_x_deltas, original_y_deltas), (stylized_image_x_deltas, stylized_image_y_deltas))

In [ ]:
show_images_with_objects([style_image, content_image, stylized_image1], titles=['Style Image', 'Content Image', 'Stylized Image'])

In [ ]:
show_images_with_objects([style_image, content_image, stylized_image1], titles=['Style Image', 'Content Image', 'Stylized Image'])

In [ ]:
import tensorflow_hub as hub

#content_image, style_image = load_images("swan-2107052_1280.jpg", style_path)

hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylized_image = hub_module(tf.image.convert_image_dtype(content_image, tf.float32), tf.image.convert_image_dtype(style_image, tf.float32))[0]
tensor_to_image(stylized_image)